# Analysis of Planck Legacy Archive Data

This notebook demonstrates how to use the CMB Analysis framework with real Planck data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cmb_analysis.cosmology import LCDM
from cmb_analysis.analysis import PowerSpectrumCalculator
from cmb_analysis.visualization import CMBPlotter
from cmb_analysis.data.planck_loader import PlanckDataLoader

%matplotlib inline
plt.style.use('seaborn-v0_8-paper')

## 1. Load Planck Data

In [ ]:
# Initialize data loader
planck = PlanckDataLoader()

# Load theoretical and observed spectra
theory_data = planck.load_theory_spectra()
observed_data = planck.load_observed_spectra()

# Get calibration factor
cal_factor = planck.get_calibration_factor()
print(f"Planck calibration factor: {cal_factor}")

## 2. Compare Theory with Data

In [ ]:
plotter = CMBPlotter()

# Prepare data for plotting
theory = {
    'cl_tt': theory_data['tt'] * cal_factor**2,
    'cl_te': theory_data['te'] * cal_factor**2,
    'cl_ee': theory_data['ee'] * cal_factor**2
}

data = {
    'cl_tt': observed_data['tt']['spectrum'],
    'cl_te': observed_data['te']['spectrum'],
    'cl_ee': observed_data['ee']['spectrum']
}

errors = {
    'cl_tt': (observed_data['tt']['error_plus'] + observed_data['tt']['error_minus'])/2,
    'cl_te': (observed_data['te']['error_plus'] + observed_data['te']['error_minus'])/2,
    'cl_ee': (observed_data['ee']['error_plus'] + observed_data['ee']['error_minus'])/2
}

# Plot comparison
fig = plotter.plot_power_spectra(theory, data, errors)
plt.show()

## 3. Compute Residuals

In [ ]:
# Plot residuals
fig = plotter.plot_residuals(theory, data, errors)
plt.show()

# Compute chi-square
chi2 = planck.compute_chi_square(theory)
print("Chi-square values:")
for spec, value in chi2.items():
    print(f"{spec.upper()}: {value:.2f}")

## 4. Parameter Estimation

In [ ]:
from cmb_analysis.analysis import MCMCAnalysis

# Set up MCMC analysis
calculator = PowerSpectrumCalculator()
mcmc = MCMCAnalysis(calculator, data, errors)

# Run MCMC
results = mcmc.run_mcmc()

# Plot parameter constraints
from cmb_analysis.visualization import MCMCDiagnostics
diagnostics = MCMCDiagnostics()

fig = diagnostics.plot_parameter_constraints(results)
plt.show()